In [1]:
import requests
import pandas
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.pinecone import Pinecone
from supabase.client import Client, create_client
import datetime
from tqdm import tqdm

import pinecone
import os
from dotenv import load_dotenv
load_dotenv()

/Users/ryandu/Desktop/iGEM/2023/cadmir/.conda/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [12]:
index_name = "cadmir"
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment=os.getenv("PINECONE_ENV")
)

In [13]:
os.getenv("PINECONE_ENV")

'asia-southeast1-gcp-free'

In [14]:
if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
      name=index_name,
      metric='cosine',
      dimension=1536  
    )
    print("created index")

In [15]:
papersDF = pandas.read_csv('csv-microRNAAN-set (1).csv')

In [16]:
embedding = OpenAIEmbeddings()

In [17]:
loader_not_works = []
no_url = []
works=[]

In [18]:
pbar = tqdm(total=len(papersDF))
for index, row in papersDF.iterrows():
    pbar.update(1)
    if index != 0:
        pbar.set_description(f"{(len(no_url) + len(loader_not_works)) / index * 100}% failure")
    # if index <= 4911:
    #     continue
    doi = row['DOI']
    # print(f"https://api.unpaywall.org/v2/{doi}?email=ryandu9221@gmail.com")
    req = requests.get(f"https://api.unpaywall.org/v2/{doi}?email=ryandu9221@gmail.com")
    if req.status_code == 200:
        if req.json().get('is_oa') == True:
            oa = req.json().get('best_oa_location')
            if oa == None:
                no_url.append(doi)
                # print("no oa " + doi)
                continue
            url = oa.get('url_for_pdf')
            if url is None:
                no_url.append(doi)
                # print("no url " + doi)
                continue
            try:
                fileloader = OnlinePDFLoader(url)
                loader = UnstructuredPDFLoader(fileloader.file_path)
                splitter = RecursiveCharacterTextSplitter(
                    chunk_size = 1000,
                    chunk_overlap  = 0,
                    length_function = len,
                    is_separator_regex = False,
                )
                chunks = loader.load_and_split(text_splitter=splitter)
                for chunk in chunks:
                    chunk.metadata["doi"] = doi
                    chunk.metadata["add_date"] = str(datetime.datetime.now().date())
                    chunk.metadata.pop("source")
                embedded = Pinecone.from_documents(chunks, embedding,index_name=index_name )
            except Exception as error:
                loader_not_works.append(doi)
                print("not work " + doi)
                print(error)
    else:
        no_url.append(doi)
    
pbar.close()

100.0% failure:   0%|          | 6/4924 [03:03<41:50:50, 30.63s/it]


KeyboardInterrupt: 

In [6]:
store = SupabaseVectorStore(embedding=embedding, client=supabase, table_name="knowledge")
store.similarity_search("miRNA",1)

APIError: {'code': 'PGRST202', 'details': 'Searched for the function public.match_documents with parameters match_count, query_embedding or with a single unnamed json/jsonb parameter, but no matches were found in the schema cache.', 'hint': None, 'message': 'Could not find the function public.match_documents(match_count, query_embedding) in the schema cache'}

# Test Cycle

In [ ]:
test_doi = "10.1038/s41584-020-0426-0"

In [ ]:
req = requests.get(f"https://api.unpaywall.org/v2/{test_doi}?email=ryandu9221@gmail.com")
url = req.json().get('best_oa_location').get('url')
fileloader = OnlinePDFLoader(url)
loader = UnstructuredPDFLoader(fileloader.file_path)
splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 500,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = False,
)
chunks = loader.load_and_split(text_splitter=splitter)
for chunk in chunks:
    chunk.metadata["doi"] = test_doi
    chunk.metadata["add_date"] = str(datetime.datetime.now().date())
    chunk.metadata.pop("source")

In [ ]:
embedded = SupabaseVectorStore.from_documents(chunks, embedding, client=supabase, table_name="knowledge")

AttributeError: 'tuple' object has no attribute 'page_content'